In [ ]:
import gc
from glob import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

import h2o
from h2o.estimators import H2OGradientBoostingEstimator
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics

import optuna
from optuna.visualization import (
    plot_contour
    , plot_edf
    , plot_intermediate_values
    , plot_optimization_history
    , plot_parallel_coordinate
    , plot_param_importances
    , plot_slice
)

import sys
sys.path.append("../utils")
from metrics import compute_recall_at4, compute_normalized_gini, compute_amex_metric

np.random.seed(2112)
pd.set_option('display.max_columns', None)

H2O_server = h2o.init(port=54321, nthreads=-1)
h2o.remove_all()

***
## load and prepare data

In [ ]:
!ls ../data/processed/dsv06

In [ ]:
train = pd.read_parquet("../data/processed/dsv06/train.parquet")
train_labels = pd.read_csv("../data/raw/train_labels.csv", index_col="customer_ID")

In [ ]:
input_feats = train.columns.tolist()
len(input_feats)

In [ ]:
train = pd.merge(train, train_labels, how="inner", left_index=True, right_index=True)
train = train.reset_index(drop=True)

del train_labels
gc.collect()

***
## model tuning


In [ ]:
%%time
# split-data
skf = StratifiedKFold(n_splits=2, random_state=2112, shuffle=True)
skf_split = list(skf.split(train, train["target"].values))

for train_idx,valid_idx in skf_split: 
    # train
    train_df = train.loc[train_idx,:].copy()
    train_df["target"] = train_df["target"].astype(str)
    train_dset = h2o.H2OFrame(
        train_df,
        column_types = {"target":"categorical"},
    )
    # valid
    valid_df = train.loc[valid_idx,:].copy()
    valid_df["target"] = valid_df["target"].astype(str)
    valid_dset = h2o.H2OFrame(
        valid_df,
        column_types = {"target":"categorical"},
    )
    break
    
print(len(train_df))
print(len(valid_df))

In [ ]:
default_params = {
    "distribution": "bernoulli",
    "learn_rate": 0.05,
    "max_depth": 5,
    "nbins": 256,
    "seed": 42,
}

In [ ]:
def train_models(model_params: dict) -> pd.DataFrame:
        
    # dataframe to store the oof predictions
    oof = valid_df[["target"]].copy()
    oof["target"] = oof["target"].astype(int)
    oof["pred"] = -1

    model = H2OGradientBoostingEstimator(**model_params)        
    model.train(
        x=input_feats, 
        y="target", 
        training_frame=train_dset,
    )
    pred = model.predict(valid_dset).as_data_frame()["p1"].values.flatten()
    oof.loc[valid_df.index,"pred"] = pred
        
    del model
    gc.collect()
        
    return oof

In [ ]:
def objective(trial):
    sampled_params = {
        # regularization
        "min_rows": trial.suggest_int("min_rows", 100, 3000, 100),
        "col_sample_rate": trial.suggest_discrete_uniform("col_sample_rate", 0.1, 0.3, 0.05),
        "sample_rate": trial.suggest_discrete_uniform("sample_rate", 0.8, 1.0, 0.05),
        "max_abs_leafnode_pred": trial.suggest_loguniform("max_abs_leafnode_pred", 1e-2, 1e2),
        "min_split_improvement": trial.suggest_loguniform("min_split_improvement", 1e-10, 1e-3),
        "pred_noise_bandwidth": trial.suggest_float("pred_noise_bandwidth", 0., 1.),
        # general parameters
        "ntrees" : trial.suggest_int("ntrees", 1000, 5000, 50),
    }    
    model_params = {**default_params, **sampled_params}
    
    oof = train_models(model_params)
    metric = compute_amex_metric(oof.target.values, oof.pred.values)
    return metric

In [ ]:
do_optimize = True

study = optuna.create_study(
    study_name="h2ogbm-dsv06",
    direction='maximize',
    storage='sqlite:///h2ogbm-dsv06.db',
    load_if_exists=True,
)

if do_optimize:
    study.optimize(
        objective, 
        n_trials=1000, 
        timeout=216000, # 2.5-days
        n_jobs=1, 
        gc_after_trial=True,
    ) 

In [ ]:
study.trials_dataframe().sort_values("value", ascending=False).head(20)

In [ ]:
plot_optimization_history(study)

In [ ]:
plot_param_importances(study)

In [ ]:
plot_slice(study)

In [ ]:
plot_edf(study)

In [ ]:
plot_parallel_coordinate(study)

In [ ]:
best_params = dict(study.best_params)
best_params = {**default_params, **best_params}
best_params

***